In [2]:
def openalex_orcid(author):
    from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto={openalex_usermail}"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'institutions':author_openalex.get('affiliations')[0].get('institutions'),
                           'institutions_years':author_openalex.get('affiliations')[0].get('years'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives')}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")                    
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")

In [39]:
import pandas as pd
df=pd.DataFrame()

In [ ]:
df

In [ ]:
import requests
author = {
"name": 'adraoui mohamed-ali',
"orcid": '0000-0001-5602-8625'
}
url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
nb_openalex=requests.get(url).json().get("meta").get('count')
if nb_openalex>0:
    for n in range(nb_openalex):
        print(n)
        author_openalex = requests.get(url).json().get("results")[n]
        result = author | {'display_name':author_openalex.get('display_name'),
                        'openalex_id':author_openalex.get('id'), 
                        'affiliations':author_openalex.get('affiliations'), 
                        'topics':author_openalex.get('topics'), 
                        'x_concepts':author_openalex.get('x_concepts'), 
                        'ids':author_openalex.get('ids'), 
                'display_name_alternatives':author_openalex.get('display_name_alternatives')}

In [ ]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [ ]:
from config_path import PATH_API
import os, pandas as pd
 

for racine, repertoires, fichiers in os.walk(PATH_API):
    print(f"{racine}, {repertoires}, {fichiers}")
    for fichier in fichiers:
        if fichier.startswith('persons'):
            print(os.path.join(racine, fichier))
            globals()[f"{fichier}"]= pd.read_pickle(os.path.join(racine, fichier))


In [ ]:
fl=['persons_author.pkl', 'persons_author_10000.pkl', 'persons_author_12000.pkl', 'persons_author_2000.pkl', 'persons_author_4000.pkl', 'persons_author_6000.pkl', 'persons_author_8000.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']
for racine, repertoires, fichiers in os.walk(PATH_API):
    for i in fl:
        name=f"{i}".split('.')[0]
        print(name)
        globals()[name] = pd.read_pickle(os.path.join(racine, fichier))

In [19]:
pers=pd.concat([persons_author_10000, persons_author_12000, persons_author_2000, persons_author_4000, persons_author_6000, persons_author_8000], ignore_index=True)

fl=['persons_author.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']

In [34]:
def openalex_orcid(author):
    # from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author

In [105]:
def openalex_name(author):
    import time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    

In [ ]:
from config_path import PATH_CLEAN
# from functions_shared import chunkify
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
# pp = pd.concat([perso_part[['contact', 'orcid_id', 'country_code']].drop_duplicates(), perso_app[['contact', 'orcid_id', 'country_code']].drop_duplicates()], ignore_index=True)
pp = perso_part[['contact', 'orcid_id', 'country_code']].fillna('')
pp=pp.loc[(pp.country_code=='FRA')].sort_values('orcid_id', ascending=False)
pp
data_chunks=list(chunkify(pp, 10000))
for i in range(0, len(data_chunks)):
    print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
    # print(type(data_chunks))
    df_temp = data_chunks[i]
    

In [ ]:
import pickle, pandas as pd
from config_path import PATH_API
pers_api=[]
for i in range(1,3):
    print(i)
    with open(f"{PATH_API}fr_persons_author_{i}.pkl", 'rb') as f:
        globals()[f"pers_api{i}"] = pickle.load(f)
    pers_api.extend(globals()[f"pers_api{i}"])


In [100]:
pers_api=pd.json_normalize(pers_api1, record_path=['affiliations'], meta=['name', 'orcid', 'display_name', 'openalex_id',  'match',  ["ids", "orcid"]],
        errors='ignore')

pers_api.columns = pers_api.columns.str.replace(r"[.*_]+", '_', regex=True)

pers_api = (pers_api
            .rename(columns={
                    'institution_country_code':'country_code'})
            .drop(columns=['institution_type','institution_lineage']))

for i in ['ids_orcid', 'institution_ror']:
    pers_api.loc[~pers_api[i].isnull(), i] = pers_api.loc[~pers_api[i].isnull()][i].str.split("/").str[-1]

In [ ]:
from config_path import PATH_CLEAN
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

In [27]:
with open(f"{PATH_API}persons_author.pkl", 'rb') as f:
    author_orcid = pickle.load(f)

In [ ]:
author_orcid=pd.json_normalize(author_orcid, record_path=['affiliations'], meta=['name','orcid', 'display_name', 'ids', 'match'])
author_orcid